In [29]:
import requests # 웹 요청 도구
from bs4 import BeautifulSoup # html에서 데이터 읽는 도구
from bs4.element import NavigableString, Tag, Comment, ProcessingInstruction

In [30]:
response = requests.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findRealTicketList.do")

In [31]:
soup = BeautifulSoup(response.content)

In [32]:
table = soup.select_one('.tbl_comm.th_sort')

In [33]:
th_list = table.select('thead tr th')
columns = []
for th in th_list:
    for c in th.children:
        # print(c)
        if type(c) == NavigableString and len(c.strip()) > 0:
            columns.append(c.strip())

In [34]:
columns

['순위', '영화명', '개봉일', '예매율', '예매매출액', '누적매출액', '예매관객수', '누적관객수']

In [35]:
tr_list = table.select('tbody tr')
values_list = []
for tr in tr_list:
    for td in tr.select('td'):
        print(td)


#beautifulsoup만으로는 크롤링 불가능. selenium(+chrome webdriver) 설치 필수

<td colspan="8">검색된 데이터가 존재하지 않습니다.</td>


In [36]:
from selenium import webdriver

In [37]:
browser_proxy = webdriver.Chrome(r"C:\tools\chromedriver")

In [38]:
browser_proxy.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findRealTicketList.do")

In [39]:
soup = BeautifulSoup(browser_proxy.page_source)

In [40]:
table = soup.select_one('.tbl_comm.th_sort')

In [41]:
th_list = table.select('thead tr th')
columns = []
for th in th_list:
    for c in th.children:
        # print(c)
        if type(c) == NavigableString and len(c.strip()) > 0:
            columns.append(c.strip())

In [42]:
columns

['순위', '영화명', '개봉일', '예매율', '예매매출액', '누적매출액', '예매관객수', '누적관객수']

In [43]:
tr_list = table.select('tbody tr')
values_list = []
for tr in tr_list:
    values = []
    for td in tr.select('td'):
        values.append(td.text.strip().replace("%", '').replace(",", ""))
    values_list.append(values)


In [44]:
values_list[:3]

[['1',
  '스파이더맨: 노 웨이 홈',
  '2021-12-15',
  '74.2',
  '2874417320',
  '0',
  '251300',
  '0'],
 ['2',
  '몬스타엑스 : 더 드리밍',
  '2021-12-08',
  '2.9',
  '158965000',
  '183940000',
  '9943',
  '11540'],
 ['3',
  '듄',
  '2021-10-20',
  '2.8',
  '140821560',
  '16458336220',
  '9469',
  '1512519']]

In [45]:
import pandas as pd

data = pd.DataFrame(values_list, columns=columns)

In [46]:
# data.info()
type(data.dtypes)
for column in data.columns[[0, 3, 4, 5, 6, 7]]:
    data[column] = data[column].astype(float)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   순위      231 non-null    float64
 1   영화명     231 non-null    object 
 2   개봉일     231 non-null    object 
 3   예매율     231 non-null    float64
 4   예매매출액   231 non-null    float64
 5   누적매출액   231 non-null    float64
 6   예매관객수   231 non-null    float64
 7   누적관객수   231 non-null    float64
dtypes: float64(6), object(2)
memory usage: 14.6+ KB


In [48]:
data.head(10)

,순위,영화명,개봉일,예매율,예매매출액,누적매출액,예매관객수,누적관객수
0,1.0,스파이더맨: 노 웨이 홈,2021-12-15,74.2,2.874417e+09,0.000000e+00,251300.0,0.0
1,2.0,몬스타엑스 : 더 드리밍,2021-12-08,2.9,1.589650e+08,1.839400e+08,9943.0,11540.0
2,3.0,듄,2021-10-20,2.8,1.408216e+08,1.645834e+10,9469.0,1512519.0
3,4.0,엔칸토: 마법의 세계,2021-11-24,2.5,8.625025e+07,4.502533e+09,8570.0,473328.0
4,5.0,연애 빠진 로맨스,2021-11-24,2.4,8.471087e+07,4.601473e+09,7966.0,453833.0
5,6.0,돈 룩 업,2021-12-08,2.3,7.510948e+07,1.812364e+08,7848.0,20204.0
6,7.0,유체이탈자,2021-11-24,1.5,5.448048e+07,6.706885e+09,5212.0,691221.0
7,8.0,마이 뉴욕 다이어리,2021-12-09,0.9,2.592850e+07,3.245014e+07,2916.0,4322.0
8,9.0,극장판 소드 아트 온라인 -프로그레시브- 별 없는 밤의 아리아,2021-12-09,0.8,3.228030e+07,5.960700e+07,2816.0,5749.0
9,10.0,티탄,2021-12-09,0.5,2.027587e+07,2.200428e+07,1806.0,2234.0


In [49]:
data.groupby(by=["순위"])["순위"].count()

순위
1.0       1
2.0       1
3.0       1
4.0       1
5.0       1
         ..
163.0     5
168.0    14
182.0    21
203.0     7
210.0    22
Name: 순위, Length: 106, dtype: int64

In [50]:
import pymysql
import pandas as pd
import os

In [55]:
# csv파일 DB(mysql)에 저장  

import numpy as np

# 1. connect
conn = pymysql.connect(host="127.0.0.1", port=3306, db='finalteam3', user="root", password="mysql", charset="utf8")

# 2. get command object
cursor = conn.cursor()

cursor.execute("delete from movie_ticketing") #기존 데이터를 지울 때 주석을 풀어주세요 

# 3. execute command
sql = """insert into movie_ticketing (ranked, title, release_date, reserve_rate, reserve_revenue, accumulated_revenue, ticketing_view, accumulated_view) 
         values (%s, %s, %s, %s, %s, %s, %s, %s)"""
for movie in data.values:
    cursor.execute(sql, list(movie))
                           
# break

conn.commit() # confirm previous execution

# 4. close resource
cursor.close()
conn.close()

OperationalError: (1205, 'Lock wait timeout exceeded; try restarting transaction')

In [ ]:
browser_proxy.close() #주석 풀어서 사용하세요) 가장 마지막 크롬 하나만 종료
browser_proxy.quit() #주석 풀어서 사용하세요) 크롬s 모두 종료